In [1]:
#|default_exp tools.sanitizer

In [2]:
#|echo: false
%load_ext autoreload
%autoreload 2

In [3]:
#|export
from typing import List, Collection
import pandas as pd
import numpy as np

import multiprocessing as mp
from chemtools.utils import convert_smiles
from rdkit import Chem
from rdkit.Chem import rdchem, rdmolops, SanitizeMol
# from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser
# from rdkit.Chem.SaltRemover import SaltRemover

from rdkit.Chem.rdchem import Atom
from rdkit.Chem.MolStandardize import rdMolStandardize

from fastprogress.fastprogress import master_bar, progress_bar
from time import sleep

from rdkit import RDLogger                                                                                                                                                               
RDLogger.DisableLog('rdApp.*')  

In [4]:
#|hide
from nbdev.showdoc import show_doc
from nbdev import nbdev_export

# Standardizer

## Helper functions

In [5]:
#|export
#_saltremover = SaltRemover()
#_unwanted = Chem.MolFromSmarts('[!#1!#5!#6!#7!#8!#9!#15!#16!#17!#35!#53]')
_allowed_atoms = [Atom(i) for i in [1,5,6,7,8,9,15,16,17,35,53]]

In [6]:
#|export
def mol_to_inchi(mol):
    """Converts a rdchem.Mol object into InCHI keys"""
    mol = convert_smiles(mol, sanitize=True)
    inchi = Chem.MolToInchiKey(mol)
    return inchi   

def build_inchi_dict(smiles_list):
    if smiles_list is None:
        print("Please provide a valid list of molecules")
    inchi_dict = {mol_to_inchi(smi):smi for smi in smiles_list}
    return inchi_dict

def find_inchikey_duplicates(mol_list):
    from collections import Counter
    
    """Analyze a list of rdkit.Mol and identify InChIKey with more than molecule.

    Arguments:
        mol_list : List rdkit.Mol objects to analyze

    Returns:
        duplis : List of duplicates
    """
    
    inchi_dict = build_inchi_dict(mol_list)
    inchi_key_list = [mol_to_inchi(mol) for mol in mol_list]
    
    if len(inchi_key_list) == len(set(inchi_key_list)):
        return {}

    counts = dict(Counter(inchi_key_list))
    duplis = [inchi_dict[inchi] for inchi in inchi_dict.keys() if counts[inchi]>1]
    return duplis

In [7]:
#|export
def add_nitrogen_charges(mol):
    """Fixes charge on nitrogen if its valence raises an Exception on RDKit
    See the discussion on: https://github.com/rdkit/rdkit/issues/3310
    """

    mol.UpdatePropertyCache(strict=False)
    ps = Chem.DetectChemistryProblems(mol)
    if not ps:
        Chem.SanitizeMol(mol)
        return mol
    for p in ps:
        if p.GetType() == 'AtomValenceException':
            at = mol.GetAtomWithIdx(p.GetAtomIdx())
            if at.GetAtomicNum() == 7 and at.GetFormalCharge() == 0 and at.GetExplicitValence() == 4:
                at.SetFormalCharge(1)
    Chem.SanitizeMol(mol)
    return mol


# def getlargestfragment(mol):
#     """Get largest fragments in a molecule"""
#     frags = rdmolops.GetMolFrags(mol, asMols=True, sanitizeFrags=True)

#     frag_size = ((idx, m.GetNumHeavyAtoms()) for idx, m in enumerate(frags) if
#                  m.GetNumHeavyAtoms() >= 6 and Chem.rdMolDescriptors.CalcNumRings(m) >= 1)
#     maxmol = sorted(frag_size, key=lambda x: x[1], reverse=True)[0][0]
#     return frags[maxmol]


def remove_unwanted(mol):
    """Remove molecules with unwanted elements (check the _unwanted definition) and isotopes"""

    remover = rdMolStandardize.AllowedAtomsValidation(_allowed_atoms)

    if len(remover.validate(mol))!=0:
        return None
    
    if sum([atom.GetIsotope() for atom in mol.GetAtoms()]) != 0:
        return None
    
    return mol

# def get_mixture(mol):
#     """Find mixtures of compounds in a rdchem.Mol object \
#     If a mixture is present, returns None

#     """
#     frags = rdmolops.GetMolFrags(mol, asMols=True, sanitizeFrags=True)
#     if len(frags) == 1:
#         return frags[0]
#     else:
#         return None


def normalize_mol(mol):
    """
    Standardize a rdchem.Mol object.
    
    Arguments:
    
        mol : rdchem.Mol
        
    
    Returns:
        mol : rdchem.Mol
            A standardized version of `mol`.
            
    Steps:
    
    1) Convert a SMILES to a rdchem.Mol object or return `mol` if it's already a rdchem.Mol
    
    2) Kekulize, check valencies, set aromaticity, conjugation and hybridization, remove hydrogens, disconnect metal atoms, normalize the molecule and reionize it. 

    3) Get parent fragment if multiple molecules (e.g. mixtures) are present
    
    4) Neutralize parent molecule
    
    
    """
    mol = convert_smiles(mol, sanitize=False) # Converts smiles into a rdchem.Mol object. If type(`mol`) = rdchem.Mol, returns `mol`.
    
 
    # Correction of nitro groups
    patt = Chem.MolFromSmarts('[O-]N(=O)')
    if mol.HasSubstructMatch(patt): 
        mol = add_nitrogen_charges(mol)
    
    mol = remove_unwanted(mol) # Remove unwanted atoms
    
    # removeHs, disconnect metal atoms, normalize the molecule, reionize the molecule
    clean_mol = rdMolStandardize.Cleanup(mol) 
     
    # if many fragments, get the "parent" (the actual mol we are interested in) 
    parent_clean_mol = rdMolStandardize.FragmentParent(clean_mol)
         
    # try to neutralize molecule
    uncharger = rdMolStandardize.Uncharger() # annoying, but necessary as no convenience method exists
    uncharged_parent_clean_mol = uncharger.uncharge(parent_clean_mol)
         
    return uncharged_parent_clean_mol

def get_stereo_info(mol):
    try:
        stereo = Chem.FindMolChiralCenters(convert_smiles(mol,sanitize=True),force=True,includeUnassigned=True,useLegacyImplementation=False)
    except RuntimeError as error:
        print(error)
        stereo = 'Stereo not calculated'
    centers =[]
    for center in stereo:
        st = '_'.join(map(str, center))
        centers.append(st)
    return '|'.join(centers)

def remove_racemic(df, smiles_col, stereo_col):    
    
    """    
    Remove records containing Mixture or Racemic information in the stereochemistry column from Dotmatics
    Compounds are only removed if the SMILES are duplicated and single isomer is also present.
    
    Arguments:
    
        df : `pandas.DataFrame`
        
            A pandas dataframe.
            
        smiles_col : str
        
            A string representing the column of `df` with SMILES.

        stereo_col : str
        
            A string representing the column of `df` with stereochemistry information for each molecule.          
    
    Returns:
        A list of columns with modifier values
        
    """
    m1 = df[smiles_col].duplicated(keep=False)
    m2 = (df[stereo_col].str.contains('|'.join(['Mixture',
                                                'mixture',
                                                'Racemic', 
                                                'racemic',
                                                'diasteroisomeric',
                                                'Diasteroisomeric'])))

    no_racemic = df[~(m1 & m2)]
    no_racemic.reset_index(drop=True, inplace=True)
    return no_racemic


def process_duplicates(data, smiles_col:str, act_col:str, id_col:str, cols_to_check:list=None, stereo_col:str='Stereo', keep:str='first'):
    """
    Removes duplicated records from a dataset
    
    The removal of duplicates is based on the stereochemical information provided by dotmatics, SMILES, ids and bioactivity records.
    Records sharing the same SMILES, IDs and/or bioactivity information, are merged by taking the median bioactivity value. 
    This step is used to clean records with multiple measurements.
    
    Duplicated records with different IDs and/or bioactivities are treated separately because they usually represent different isomers. 
    For these compounds, the duplicated records are first split into `to_merge` and `to_keep` portions, indicating the different compounds
    should be considered to be kept in the dataset or merged into the same record, respectively. Duplicated records are considered for keeping,
    if the difference between max and min bioactivity is larger or equal to 1.0 (e.g. 1 pIC50 unit); else the records are considered for merging.
    
    Compounds marked for keeping filtered by the `keep` parameter. If `keep == True`, the duplicate with highest bioactivity value is kept; else the duplicate with lowest bioactivity is kept.
    Compounds marked for merging are combined by taking the median of all columns.
    
    Arguments:
    

            data : pd.DataFrame
                A pandas Dataframe with molecular data for sanitization.

            smiles_col : str
                The name of the column with SMILES for each molecule.

            id_col : str
                The name of the column with IDs for each molecule.
                
            act_col : str
                The name of the column with bioactivity data for each molecule. 
                
                
            cols_to_check : list
                A list of columns to use as reference to find duplicates.
                

        Returns:
            recombined_data : `pandas.DataFrame`

                A sanitized version of the `data` without duplicates.   
    
    
    
    """
    assert keep in ['first','last']
    data = data.copy()   
    stereo_col = 'Stereo'
    
    # Convert to Inchi
    data['inchi'] = data[smiles_col].apply(mol_to_inchi)
    data[stereo_col] = data[smiles_col].apply(lambda x : get_stereo_info(x))
    #data = remove_racemic(df=data, smiles_col=smiles_col,stereo_col=stereo_col)
    
    if cols_to_check:
        data['duplicate'] = data.duplicated(subset=['inchi']+cols_to_check, keep=False).astype(bool)
    else: 
        data['duplicate'] = data.duplicated(subset=['inchi'], keep=False).astype(bool)
    duplicates = data[data['duplicate']==True]
    no_duplicates = data[data['duplicate']==False]
    no_duplicates.reset_index(drop=True,inplace=True)
    
    if duplicates.empty == False:
        # Remove duplicates
        processed_duplicates = [] # List to keep processed duplicates 

        ### Same ID, same SMILES  
        duplis_grouped = duplicates.groupby([id_col, 'inchi'], as_index=False).median().reset_index(drop=True)
        duplis_grouped.reset_index(drop=True,inplace=True)
        duplicates.reset_index(drop=True,inplace=True)

        real_duplis_removed = duplis_grouped.merge(duplicates.drop_duplicates(subset=[id_col,
                                                                                        'inchi',
                                                                                        act_col],
                                                                                keep='first').reset_index(drop=True),
                                                 on=[id_col, 'inchi'],
                                                 how='outer', 
                                                 suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

        ### Different ID, same SMILES, same activity

        ### Different ID, same SMILES
        # Combine duplicates with same SMILES but different IDS and/or bioactivities
        groups = real_duplis_removed.groupby('inchi')[act_col].apply(lambda x : np.abs(np.max(x) - np.min(x))).reset_index().dropna(subset=[act_col])

        # Split duplicates by type. If the difference in bioactivities is > 1.0, the records should be kept; else the records will be merged
        to_keep = real_duplis_removed[real_duplis_removed['inchi'].isin(groups[groups[act_col].ge(1.0)]['inchi'].unique())].reset_index(drop=True)
        to_merge = real_duplis_removed[real_duplis_removed['inchi'].isin(groups[groups[act_col].lt(1.0)]['inchi'].unique())].reset_index(drop=True)

        if to_merge.empty == False:
            # Merging duplicates
            to_merge[id_col] = to_merge.groupby('inchi')[id_col].transform('first')

            to_merge_median = to_merge.groupby([id_col,'inchi'],as_index=False).median().reset_index(drop=True)

            # Recombine information to merged data
            duplis_merged = to_merge_median.merge(to_merge.drop_duplicates(subset=['inchi'],keep='first'), on=[id_col,'inchi'],how='outer', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
            duplis_merged.reset_index(drop=True,inplace=True)
            processed_duplicates.append(duplis_merged)

        else:
            print('No duplicates to merge')


        # Keeping duplicates
        if to_keep.empty == False and keep == 'first':

            duplis_kept = to_keep.groupby('inchi', group_keys=False).apply(lambda x: x.loc[x[act_col].idxmax()]).copy().reset_index(drop=True)
            duplis_kept.reset_index(drop=True,inplace=True)
            processed_duplicates.append(duplis_kept)

        elif to_keep.empty == False and keep == 'last':
            duplis_kept = to_keep.groupby('inchi', group_keys=False).apply(lambda x: x.loc[x[act_col].idxmin()]).copy().reset_index(drop=True)
            duplis_kept.reset_index(drop=True,inplace=True)        
            processed_duplicates.append(duplis_kept)

        else:
            print('No duplicates to keep')
            
        # Recombine data
        return pd.concat([no_duplicates, *processed_duplicates],axis=0)

    else:
        return no_duplicates



## MolCleaner

In [8]:
#|export
class MolCleaner:
    
    """Sanitize a molecular dataset
    
        Use static methods `MolCleaner.from_df or MolCleaner.from_csv` instead of accessing the \
        class directly.
        
        1. Standardize unknown stereochemistry (Handled by the RDKit Mol file parser)
            i) Fix wiggly bonds on sp3 carbons - sets atoms and bonds marked as unknown stereo to no stereo
            ii) Fix wiggly bonds on double bonds – set double bond to crossed bond
        2. Clears S Group data from the mol file
        3. Kekulize the structure
        4. Remove H atoms (See the page on explicit Hs for more details)
        5. Normalization:
            Fix hypervalent nitro groups
            Fix KO to K+ O- and NaO to Na+ O- (Also add Li+ to this)
            Correct amides with N=COH
            Standardise sulphoxides to charge separated form
            Standardize diazonium N (atom :2 here: [*:1]-[N;X2:2]#[N;X1:3]>>[*:1]) to N+
            Ensure quaternary N is charged
            Ensure trivalent O ([*:1]=[O;X2;v3;+0:2]-[#6:3]) is charged
            Ensure trivalent S ([O:1]=[S;D2;+0:2]-[#6:3]) is charged
            Ensure halogen with no neighbors ([F,Cl,Br,I;X0;+0:1]) is charged
        6. The molecule is neutralized, if possible. See the page on neutralization rules for more details.
        7. Remove stereo from tartrate to simplify salt matching
        8. Normalise (straighten) triple bonds and allenes
        
             
        The curation steps in ChEMBL structure pipeline were augmented with additional steps to identify duplicated entries
        9. Find stereo centers
        10. Generate inchi keys
        11. Find duplicated SMILES. If the same SMILES is present multiple times, two outcomes are possible.
            i. The same compound (e.g. same ID and same SMILES)
            ii. Isomers with different SMILES, IDs and/or activities
            
            In case i), the compounds are merged by taking the median values of all numeric columns in the dataframe. 
            For case ii), the compounds are further classified as 'to merge' or 'to keep' depending on the activity values.
                a) Compounds are considered for mergining (to merge) if the difference in acvitities is less than 1log unit.
                b) Compounds are considered for keeping as individual entries (to keep) if the difference in activities is larger than 1log unit. In this case, the user can
                select which compound to keep - the one with highest or lowest activity.
        
       
    """

    @classmethod
    def process_mol(cls, mol):
        """Fully process one molecule"""
        mol = convert_smiles(mol)

        try:
            # Remove salts and molecules with unwanted elements (See _allowed_atoms definition above)
            mol = normalize_mol(mol)


            # Check if mixture is present - standard advice is to remove all mixtures
            #mol = get_mixture(mol) # Deprecated since this step already happens in `normalize_mol` using `rdMolStandardize.FragmentParent`

            if isinstance(mol, rdchem.Mol):
                return Chem.MolToSmiles(mol)
        except:

            return None

    @classmethod
    def process_smiles_list(cls, smiles_list):
        """Sanitize list of SMILES"""

        #sanitized_smiles = map(cls.process_mol, tqdm(smiles_list, desc='Sanitizing SMILES', position=0, leave=True))
        with mp.Pool() as mp_pool:

            sanitized_smiles = list(progress_bar(mp_pool.imap(cls.process_mol, smiles_list), total=len(smiles_list)))
            
        # failed_smiles = set.difference(set(smiles_list), set(sanitized_smiles))
        # with open('./failed_mols', 'w') as f:
        #     for s in failed_smiles:
        #         f.write(s+'\n')
        return sanitized_smiles



    @classmethod
    def from_list(cls, smiles_list):
        
        """Factory method to process a collection of SMILES

        Arguments:

            smiles : A List, Array, or any Iterable (except strings)
                SMILES ready for sanitization

        Returns:
            processed_mols : `pandas.DataFrame`

                A sanitized version of the `df` including all molecules. If sanitization fails for a molecule, \
                the original SMILES is kept but `processed_smiles` field will be NaN

        """
        act_col = 'Target'
        id_col = 'ID'
        smiles_col = 'SMILES'

        df = pd.DataFrame({smiles_col:smiles_list, 
                           id_col:[f'mol{idx}' for idx in range(len(smiles_list))],
                          act_col:[-9999]*len(smiles_list)})


        return cls.from_df(df, 
                           smiles_col=smiles_col,
                           id_col=id_col, 
                           act_col=act_col)


    @classmethod
    def from_df(cls,
                df: pd.DataFrame, 
                smiles_col: str,
                act_col: str,
                id_col: str,
                keep:str='first',
                cols_to_check:List=None):
        
        """Factory method to process a `pandas.DataFrame`

        Arguments:

            df : pd.DataFrame
                A pandas Dataframe with molecular data for sanitization.

            smiles_col : str
                The name of the column with SMILES for each molecule.

            id_col : str
                The name of the column with IDs for each molecule. If None, generates random IDs.

        Returns:
            processed_mols : `pandas.DataFrame`

                A sanitized version of the `df` including all molecules. If sanitization fails for a molecule,
                the original SMILES is kept but `processed_smiles` field will be NaN

        """     
        

        
        data = df.copy()
        data.reset_index(drop=True,inplace=True)
        id_col = id_col
        smiles_col = smiles_col
        act_col = act_col
        processed_smiles_col = f'processed_{smiles_col}'

        data[id_col] = data[id_col].astype(str)


        sanitized_smiles = cls.process_smiles_list(data[smiles_col].values)
        
        # sanitized_smiles = cls.process_smiles_list(smiles=data[smiles_col].values,
        #                             id_col=data[id_col].values,
        #                             act_col=data[act_col].values,
        #                             keep=keep,
        #                             cols_to_check=cols_to_check)
        
        data.insert(loc=1, column=processed_smiles_col, value=list(sanitized_smiles))


        data.dropna(subset=[processed_smiles_col, act_col],inplace=True)
        data.reset_index(drop=True,inplace=True)

        # Remove duplicates
        _data = process_duplicates(data,
                                  smiles_col=processed_smiles_col,
                                  act_col=act_col,
                                  id_col=id_col,
                                  cols_to_check=cols_to_check)
        
        return _data
    

    @classmethod
    def from_csv(cls,
                 data_path: str,
                 smiles_col: str,
                 act_col:str,
                id_col: str,
                 sep: str = ',',
                keep:str='first',
                        cols_to_check:list=None):
        
        """Factory method to process a CSV file

        Arguments:

            data_path : str
                Path to CSV file

            smiles_col : str
                The name of the column with SMILES for each molecule.

            id_col : str
                The name of the column with IDs for each molecule. 
                
            id_col : str
                The name of the column with stereochemistry information for each molecule. 
 
        Returns:
            processed_mols : `pandas.DataFrame`
            

                A sanitized version of the `df` including all molecules. If sanitization fails for a molecule,
                the original SMILES is kept but `processed_smiles` field will be NaN       
        
        """
        return cls.from_df(pd.read_csv(data_path, sep=sep), 
                           smiles_col=smiles_col,
                           id_col=id_col, 
                           act_col=act_col,
                           keep=keep,
                           cols_to_check=cols_to_check)

In [9]:
#|hide
nbdev_export()